Library Download

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
import requests
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
import time
from options import Options

driver = webdriver.Chrome(executable_path="D:\Python\Driver\Chrome\chromedriver.exe")
url ="https://www.google.it/maps/place/Anteraja/@-6.224705,106.8278741,17z/data=!3m1!4b1!4m5!3m4!1s0x2e69f302656f37b5:0xe80bb4258dbec662!8m2!3d-6.224705!4d106.8300628"
driver.get(url)

In [2]:
def click_review(x):    
    wait = WebDriverWait(driver, 10)
    menu_bt = wait.until(EC.element_to_be_clickable(
                       (By.XPATH, '//button[@data-value=\'Urutkan\']'))
                   )  
    menu_bt.click()
    recent_rating_bt = driver.find_elements_by_xpath(
                                     '//li[@role=\'menuitemradio\']')[x]
    recent_rating_bt.click()
    time.sleep(5)

click_review(1)

In [11]:
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//div[@class=\'section-layout section-scrollbox cYB2Ge-oHo7ed cYB2Ge-ti6hGc\']'))) # Waits for the page to load.
pause_time = 2 # Waiting time after each scroll.
max_count = 650 # Number of times we will scroll the scroll bar to the bottom.
x = 0

while(x<max_count):
    scrollable_div = driver.find_element_by_css_selector('div.section-layout.section-scrollbox.cYB2Ge-oHo7ed.cYB2Ge-ti6hGc') # It gets the section of the scroll bar.
    try:
        driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div)# Scroll it to the bottom.
    except:
        pass
    time.sleep(pause_time) # wait for more reviews to load.
    x=x+1

KeyboardInterrupt: 

In [5]:
from bs4 import BeautifulSoup

username = []
review_text = []
rating = []
rel_date = []

response = BeautifulSoup(driver.page_source, 'html.parser')
rlist = response.find_all('div', class_='ODSEW-ShBeI NIyLF-haAclf gm2-body-2')

def scrape():
    global response
    global rlist
    response = BeautifulSoup(driver.page_source, 'html.parser')
    rlist = response.find_all('div', class_='ODSEW-ShBeI NIyLF-haAclf gm2-body-2')

In [6]:
def get_data():
   global username
   global review_text
   global rating
   global rel_date

   for r in rlist:
      username.append(r.find('div', 
                  class_='ODSEW-ShBeI-RWgCYc ODSEW-ShBeI-RWgCYc-SfQLQb-BKD3ld').find('span').text) 
      try:
         review_text.append(r.find('span', class_='ODSEW-ShBeI-text').text) 
      except Exception:
         review_text.append(None)
      rating.append(r.find('span', class_='ODSEW-ShBeI-H1e3jb')['aria-label']) 
      rel_date.append(r.find('span', class_='ODSEW-ShBeI-RgZmSc-date').text)  
get_data()

In [7]:
len(rating)

930

In [10]:
click_review(3)

In [12]:
scrape()
get_data()
len(rating)

1860

In [13]:
import numpy as np
df = pd.DataFrame(np.column_stack([username, rating, rel_date,review_text]), 
                               columns=['Nama Pengguna', 'Rating', 'Tanggal','Review'])
df

,Nama Pengguna,Rating,Tanggal,Review
0,Heru Ahyarudin,1 bintang,1 jam lalu,sudah 4 hari paket belum datang juga\nestimasi...
1,Theressia Arlenne,1 bintang,2 jam lalu,
2,Jumiani,1 bintang,3 jam lalu,
3,Bagus Setiyanto,1 bintang,4 jam lalu,"Yg lain 2-3 hari sampe,ini 3-4 hari gak sampe2..."
4,Enju Juhana,1 bintang,6 jam lalu,AnterAja apa Entaraja????
...,...,...,...,...
1855,Arga Adi,1 bintang,4 bulan lalu,"Beli barang via tokped iseng"" pake anter aja,e..."
1856,Ngilo Store,1 bintang,4 bulan lalu,Enggak lagi lagi pake jasa pengiriman ini. Dr ...
1857,Ariya Vindhika,1 bintang,4 bulan lalu,jakarta - tangerang butuh 5 hari sudah beberap...
1858,Eka Fenny Setiawan,1 bintang,4 bulan lalu,Anteraja NEXT DAY CACAT!!!!\nkalo gak bisa on ...


In [14]:
df.Review.str.split(expand=True).stack().value_counts()

di           899
barang       539
ini          499
saya         497
ke           461
            ... 
uninstall      1
mnding         1
interview      1
WIB            1
under          1
Length: 7457, dtype: int64

In [18]:
df.to_csv('D:/ITB/Lomba PPM/PPM anteraja/maps_scrape.csv')

In [27]:
df_review = df.Review.str.split(expand=True).stack().value_counts()

183

In [29]:
df["Rating"].value_counts()

 1 bintang     1649
 5 bintang       95
 2 bintang       75
 3 bintang       22
 4 bintang       19
Name: Rating, dtype: int64